## Импорт модулей

In [ ]:
import pandas as pd

In [ ]:
import sys
sys.path.append(r'E:\Git')
import time

In [ ]:
import preproc_p.preproc_tool as preproc_tool
import preproc_p.workflow_cs_data as workflow_cs_data
import plot_workflow.plotly_workflow as plotly_workflow

In [ ]:
import preproc_p.workflow_periodic_wells as workflow_periodic_wells

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import datetime
from multiprocessing import Pool
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from preproc_p import workflow_periodic_wells

In [ ]:
gn = global_names = preproc_tool.GlobalNames()

## Парсинг экселей

In [ ]:
files = preproc_tool.find_full_path_by_pattern(r'E:\Данные\2020_04_ноябрьск_су\data_sort\elr\эксели_авто', "*.*",'.xls')

In [ ]:
len(files)

In [ ]:
path_to_save = r'E:\Данные\2020_04_ноябрьск_су\data_sort\elr\эксели_авто\csv'

In [ ]:
#парсинг файликов
for j, i in enumerate(files):
    print(j)
    try:
        print(i)
        this_file = workflow_cs_data.parse_cs_data_all_types(i)
        new_file_name = path_to_save + '\\' + i.split('\\')[-1].replace('xls', 'csv')
        this_file.to_csv(new_file_name)
        print('\n')
    except:  
        print('!!!!')
        print(f"Не распарсился файл {i}")
        print('!!!!')
        print('\n')
    

## Построение графиков

In [ ]:
files_to_plot = preproc_tool.find_full_path_by_pattern(r'E:\Данные\2020_04_ноябрьск_су\data_sort\elr\эксели_авто\csv', '*.*','csv')

In [ ]:
for i in files_to_plot:
    for j in files:
        if i.split('\\')[-1].replace('.csv','') in j.split('\\')[-1]:
            print(i)
            this_df = pd.read_csv(i, index_col = [0], parse_dates = True)
            #this_df.index = pd.to_datetime(this_df.index, format="%Y-%m-%d %H:%M:%S")
            banches = plotly_workflow.create_banches_for_report(this_df, this_df.columns)
            plotly_workflow.create_report_html(this_df, banches, i + '.html', auto_open = False)

## Анализ подготовленных данных .csv

In [ ]:
files = preproc_tool.find_full_path_by_pattern(r'E:\Данные\2020_04_ноябрьск_су\dataset_to_use\04_14_2020', "*.*",'.csv')

In [ ]:
parameters = [ gn.q_liq_m3day, gn.q_oil_mass_tday, gn.q_gas_m3day,
              gn.active_power_kwt, gn.i_a_motor_a, gn.motor_load_perc, gn.freq_hz, gn.cos_phi_d,
             gn.p_intake_atm, gn.p_lin_atm, gn.t_intake_c, gn.t_motor_c,
              gn.vibration_xy_msec2, gn.vibration_z_msec2,
             "Рабочая доля времени", gn.work_status_number]

In [ ]:
def analyse_cs_data(this_file_name):
    
        df = pd.read_csv(this_file_name, index_col = [0], parse_dates = True, dayfirst = True)
        df = preproc_tool.rename_columns_by_dict(df)

        gas_borders, gas_dfs, stats, df = workflow_periodic_wells.find_gas_periods(df, gn.i_a_motor_a)

        stacks = workflow_periodic_wells.find_stucks(df)
        stats['Количество перегрузок'] = len(stacks)


        borders = [gas_borders, stacks]


        banches = pltl_wf.create_banches_for_report(df, parameters, fuzzy_names=True)
        new_file_name = this_file_name.replace('.csv', '.html')
        new_file_name = new_file_name.replace('dataset_to_use', 'analysis_result')
        print(' ---> ' + new_file_name +  '\n')


        table = pd.DataFrame(stats, index = [this_file_name.split('\\')[-1]])

        table_for_plot = table.T
        table_for_plot.index.name = 'Название исходного файла'

        borders = [gas_borders, stacks]
        pltl_wf.create_report_html(df, banches,new_file_name , borders = borders, auto_open=False, 
                                   df_for_table = table_for_plot)
        return table


    
def save_result_excel(result_file, file_name):
    new_file_name = file_name.replace('.csv', '.html')
    new_file_name = new_file_name.replace('dataset_to_use', 'analysis_result')
    excel_file_name = new_file_name
    excel_file_name = excel_file_name.replace(new_file_name.split('\\')[-1], 'result_analysis.xlsx')
    result_file.to_excel(excel_file_name)
    print(excel_file_name)
    
def run_calculation(to_mp, func, amount_of_threads):
    start_time = time.time()
    if __name__ == '__main__':
        with Pool(amount_of_threads) as p:
            result = p.map(func, to_mp)
            
    result_df  = None
    for i,j in enumerate(result):
        if i !=0 and type(j) != type(None):
            result_df = result_df.append(j)
        elif i ==0 and type(j) != type(None):
            result_df = j.copy()
        else:
            pass
    end_time = time.time()
    print(f"Затрачено времени {(end_time-start_time)/60} минут")
    return result_df
    

In [ ]:
result_file = None
for k, this_file_name in enumerate(files[:2]):
    print(this_file_name)
    
    table = analyse_cs_data(this_file_name)
    if k != 0:
        result_file = result_file.append(table)
    else:
        result_file = table.copy()


save_result_excel(result_file, files[0])

In [ ]:
result_df = run_calculation(files, workflow_periodic_wells.analyse_cs_data, 10)
save_result_excel(result_df, files[0])